In [ ]:
!pip install elasticsearch

In [ ]:
pip -q install eland elasticsearch sentence_transformers transformers torch==1.11

In [ ]:
from elasticsearch import Elasticsearch, helpers # Import the Elasticsearch client and helpers module
from urllib.request import urlopen # library for opening URLs
import json # module for handling JSON data
from pathlib import Path # module for working with file paths
# Python client and toolkit for machine learning in Elasticsearch
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel
from elasticsearch.client import MlClient # Elastic module for ml
import getpass # handling password input

In [4]:
# Found in the 'Manage Deployment' page
CLOUD_ID = getpass.getpass('Enter Elastic Cloud ID:  ')

# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = getpass.getpass('Enter Elastic password:  ')

# Create the client instance
client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=("elastic", ELASTIC_PASSWORD),
    request_timeout=3600
)

Enter Elastic Cloud ID:   ········
Enter Elastic password:   ········


In [ ]:
# Set the model name from Hugging Face and task type
# sentence-transformers model
hf_model_id='sentence-transformers/all-mpnet-base-v2'
tm = TransformerModel(hf_model_id, "text_embedding")

#set the modelID as it is named in Elasticsearch
es_model_id = tm.elasticsearch_model_id()

# Download the model from Hugging Face
tmp_path = "models"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
model_path, config, vocab_path = tm.save(tmp_path)

# Load the model into Elasticsearch
ptm = PyTorchModel(client, es_model_id)
ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config)

# Start the model
s = MlClient.start_trained_model_deployment(client, model_id=es_model_id)
s.body

In [ ]:
#Index to load products-ecommerce.json docs

client.indices.create(
    index="ecommerce",
    mappings= {
    "properties": {
      "product": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "description": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "category": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      }
  }
})

In [ ]:
# Creating an ingest pipeline with inference processors to use ELSER (sparse) and all-mpnet-base-v2 (dense) to infer against data that will be ingested in the pipeline.

client.ingest.put_pipeline( 
    id="ecommerce-pipeline",
    processors = [
    {
      "inference": {
        "model_id": ".elser_model_1",
        "target_field": "ml",
        "field_map": {
          "description": "text_field"
        },
        "inference_config": {
          "text_expansion": { # text_expansion inference type (ELSER)
            "results_field": "tokens"
          }
        }
      }
    },
    {
      "inference": {
        "model_id": "sentence-transformers__all-mpnet-base-v2",
        "target_field": "description_vector", # Target field for the inference results
        "field_map": {
          "description": "text_field" # Field matching our configured trained model input. Typically for NLP models, the field name is text_field.
        }
      }
    }
  ]
)

In [ ]:
# Reindex dest index

INDEX = 'ecommerce-search'
client.indices.create(
            index=INDEX,
            settings={
                "index": {
                    "number_of_shards": 1,
                    "number_of_replicas": 1
                }
            },
            mappings={
# Saving disk space by excluding the ELSER tokens and the dense_vector field from document source.
# Note: That should only be applied if you are certain that reindexing will not be required in the future.
            "_source" : {
            "excludes": ["ml.tokens","description_vector.predicted_value"]
           }, 
        "properties": {
        "product": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "description": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "category": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "ml.tokens": { # The name of the field to contain the generated tokens.
        "type": "rank_features" # ELSER output must be ingested into a field with the rank_features field type.
      }, 
     "description_vector.predicted_value": { # Inference results field, target_field.predicted_value
     "type": "dense_vector", 
     "dims": 768, # The all-mpnet-base-v2 model has embedding_size of 768, so dims is set to 768.
     "index": "true", 
     "similarity": "dot_product" #  When indexing vectors for approximate kNN search, you need to specify the similarity function for comparing the vectors.
 }
  }
 
}
)

In [ ]:
# Ecommerce dataset

url = "https://raw.githubusercontent.com/priscillaparodi/dataset/main/products-ecommerce.json"

response = urlopen(url)

# Load the response data into a JSON object
data_json = json.loads(response.read())

def create_index_body(doc):
    """ Generate the body for an Elasticsearch document. """
    return {
        "_index": "ecommerce",
        "_source": doc,
    }

# Prepare the documents to be indexed
documents = [create_index_body(doc) for doc in data_json]

# Use helpers.bulk to index
helpers.bulk(client, documents)

print("Done indexing documents into `ecommerce` index")

In [ ]:
# Reindex data from one index 'source' to another 'dest' with the 'ecommerce-pipeline' pipeline.

client.reindex(wait_for_completion=True,
               source={
                  "index": "ecommerce"
    },
               dest= {
                  "index": "ecommerce-search",
                  "pipeline": "ecommerce-pipeline"
    }
)

In [7]:
# BM25

print(f"\nBM25:\n")

response1 = client.search(size=2,
    index="ecommerce-search",
    query= {
            "match": {
                    "description" : {  
                    "query": "Furniture for a large balcony"
                        }
                    }
        }
)
hits = response1['hits']['hits']

if not hits:
    print("No matches found")
else:
    for hit in hits:
        score = hit['_score']
        product = hit['_source']['product']
        category = hit['_source']['category']
        description = hit['_source']['description']
        print(f"\nScore: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n")


BM25:


Score: 7.188758
Product: Scratching Post
Category: Dog and Cat Supplies & Toys
Description: is a post covered in sisal or carpet material for your cat to scratch, saving your furniture from their claws.


Score: 6.3329334
Product: Barbie Dreamhouse
Category: Toys
Description: is a classic Barbie playset with multiple rooms, furniture, and accessories. It allows kids to create their dream Barbie world.



In [8]:
# KNN

print(f"\nKNN:\n")

response2 = client.search(index='ecommerce-search', size=2,
            knn={
            "field": "description_vector.predicted_value",
            "k": 50,
            "num_candidates": 2495,
            "query_vector_builder": {
            "text_embedding": { 
            "model_id": "sentence-transformers__all-mpnet-base-v2", 
            "model_text": "Furniture for a large balcony" 
      }
    }
            }
)

for hit in response2['hits']['hits']:
    
    score = hit['_score']
    product = hit['_source']['product']
    category = hit['_source']['category']
    description = hit['_source']['description']
    print(f"\nScore: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n")


KNN:


Score: 0.7607467
Product: Resin Wicker Patio Sectional Set
Category: Outdoor Furniture
Description: is a versatile and modular patio sectional set, allowing for various seating arrangements and configurations.


Score: 0.75559103
Product: Patio Sofa Set with Ottoman
Category: Outdoor Furniture
Description: is a versatile and comfortable patio sofa set, including a sofa, ottoman, and coffee table, great for outdoor lounging.



In [9]:
# ELSER

print(f"\nELSER:\n")

response3 = client.search(index='ecommerce-search', size=2,
              query={
                  "text_expansion": {
                  "ml.tokens": {
                      "model_id":".elser_model_1",
                      "model_text":"Furniture for a large balcony"                
        }
    }
}
)

for hit in response3['hits']['hits']:

    score = hit['_score']
    product = hit['_source']['product']
    category = hit['_source']['category']
    description = hit['_source']['description']
    print(f"\nScore: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n")


ELSER:


Score: 12.1684
Product: Rattan Patio Conversation Set
Category: Outdoor Furniture
Description: is a stylish and comfortable outdoor furniture set, including a sofa, two chairs, and a coffee table, all made of durable rattan material.


Score: 11.867347
Product: Garden Lounge Set with Side Table
Category: Garden Furniture
Description: is a comfortable and stylish garden lounge set, including a sofa, chairs, and a side table for outdoor relaxation.



In [10]:
# BM25 + KNN

print(f"\nHybrid Search\n")

print(f"\nBM25 + KNN:\n")

response4 = client.search(index='ecommerce-search', size=2,
              query={
             "match": {
             "description" : {  
             "query": "Furniture for a large balcony",
              "boost": 1
                        }
                        }                 
                   
},
            knn={
            "field": "description_vector.predicted_value",
            "k": 50,
            "num_candidates": 2495,
            "boost": 1,
            "query_vector_builder": {
            "text_embedding": { 
            "model_id": "sentence-transformers__all-mpnet-base-v2", 
            "model_text": "Furniture for a large balcony" 
      }
    }
            }
)

for hit in response4['hits']['hits']:

    score = hit['_score']
    product = hit['_source']['product']
    category = hit['_source']['category']
    description = hit['_source']['description']
    print(f"\nScore: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n")



Hybrid Search


BM25 + KNN:


Score: 7.188758
Product: Scratching Post
Category: Dog and Cat Supplies & Toys
Description: is a post covered in sisal or carpet material for your cat to scratch, saving your furniture from their claws.


Score: 6.794809
Product: Rattan Patio Conversation Set
Category: Outdoor Furniture
Description: is a stylish and comfortable outdoor furniture set, including a sofa, two chairs, and a coffee table, all made of durable rattan material.



In [11]:
# BM25 + KNN (RRF)

print(f"\nHybrid Search\n")

print(f"\nBM25 + KNN (RRF):\n")

response5 = client.search(index='ecommerce-search', size=2,
             query = {
             "match": {
             "description" : {  
             "query": "Furniture for a large balcony",
                        }
                        }                 
                   
},
             knn = {
            "field": "description_vector.predicted_value",
            "k": 50,
            "num_candidates": 2495,
            "query_vector_builder": {
            "text_embedding": { 
            "model_id": "sentence-transformers__all-mpnet-base-v2", 
            "model_text": "Furniture for a large balcony" 
      }
    }
            },
        rank = {
        "rrf": {
            "window_size": 2495,
            "rank_constant": 10
        }
    }
)

for hit in response5['hits']['hits']:
    
    rank = hit['_rank']
    category = hit['_source']['category']
    product = hit['_source']['product']
    description = hit['_source']['description']
    print(f"\nRank: {rank}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n")


Hybrid Search


BM25 + KNN (RRF):


Rank: 1
Product: Rattan Patio Conversation Set
Category: Outdoor Furniture
Description: is a stylish and comfortable outdoor furniture set, including a sofa, two chairs, and a coffee table, all made of durable rattan material.


Rank: 2
Product: Resin Wicker Patio Sectional Set
Category: Outdoor Furniture
Description: is a versatile and modular patio sectional set, allowing for various seating arrangements and configurations.



In [12]:
# BM25 + ELSER
# Note: Client is not compatible with Elastic 8.9 to use sub_searches parameter. Not adding RRF, for now.
# Using compound query - 'should' appear in the matching document.

print(f"\nHybrid Search\n")

print(f"\nBM25 + ELSER:\n")

response6 = client.search(index='ecommerce-search', size=2,

        query= {
            "bool": {
                "should": [
                    {
                        "match": {
                            "description" : {  
                            "query": "Furniture for a large balcony",
                            "boost": 1
                        }
                        }
                    },                   
                    {
                        "text_expansion": {
                            "ml.tokens": {
                                "model_id": ".elser_model_1",
                                "model_text": " Furniture for a large balcony",
                                "boost": 1
                            }
                        }
                    }
                ]
            }
        }

)

for hit in response6['hits']['hits']:

    score = hit['_score']
    product = hit['_source']['product']
    category = hit['_source']['category']
    description = hit['_source']['description']
    print(f"\nScore: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n")


Hybrid Search


BM25 + ELSER:


Score: 18.218067
Product: Rattan Patio Conversation Set
Category: Outdoor Furniture
Description: is a stylish and comfortable outdoor furniture set, including a sofa, two chairs, and a coffee table, all made of durable rattan material.


Score: 13.042423
Product: Garden Lounge Set with Side Table
Category: Garden Furniture
Description: is a comfortable and stylish garden lounge set, including a sofa, chairs, and a side table for outdoor relaxation.

